## Tutorial 07 - Non linear Elliptic problem
**_Keywords: DEIM, POD-Galerkin_**

### 1. Introduction

In this tutorial, we consider a non linear elliptic problem in a two-dimensional spatial domain $\Omega=(0,1)^2$. We impose a homogeneous Dirichlet condition on the boundary $\partial\Omega$. The source term is characterized by the following expression
$$
g(\boldsymbol{x}; \boldsymbol{\mu}) = 100\sin(2\pi x_0)cos(2\pi x_1) \quad \forall \boldsymbol{x} = (x_0, x_1) \in \Omega.
$$

This problem is characterized by two parameters. The first parameter $\mu_0$ controls the strength of the sink term and the second parameter $\mu_1$ the strength of the nonlinearity. The range of the two parameters is the following:
$$
\mu_0,\mu_1\in[0.01,10.0]
$$
The parameter vector $\boldsymbol{\mu}$ is thus given by
$$
\boldsymbol{\mu} = (\mu_0,\mu_1)
$$
on the parameter domain
$$
\mathbb{P}=[0.01,10]^2.
$$


In order to obtain a faster approximation of the problem, we pursue a model reduction by means of a POD-Galerkin reduced order method. In order to preserve the affinity assumption the discrete empirical interpolation method will be used on the forcing term $g(\boldsymbol{x}; \boldsymbol{\mu})$.



### 2. Parametrized formulation

Let $u(\boldsymbol{\mu})$ be the solution in the domain $\Omega$.

The strong formulation of the parametrized problem is given by: for a given parameter $\boldsymbol{\mu}\in\mathbb{P}$, find $u(\boldsymbol{\mu})$ such that

$$ -\nabla^2u(\boldsymbol{\mu})+\frac{\mu_0}{\mu_1}(\exp\{\mu_1u(\boldsymbol{\mu})\}-1)=g(\boldsymbol{x}; \boldsymbol{\mu})$$
<br>
    
The corresponding weak formulation reads: for a given parameter $\boldsymbol{\mu}\in\mathbb{P}$, find $u(\boldsymbol{\mu})\in\mathbb{V}$ such that

$$a\left(u(\boldsymbol{\mu}),v;\boldsymbol{\mu}\right)+c\left(u(\boldsymbol{\mu}),v;\boldsymbol{\mu}\right)=f(v;\boldsymbol{\mu})\quad \forall v\in\mathbb{V}$$

where

* the function space $\mathbb{V}$ is defined as
$$
\mathbb{V} = \{v\in H_1(\Omega) : v|_{\partial\Omega}=0\}
$$
* the parametrized bilinear form $a(\cdot, \cdot; \boldsymbol{\mu}): \mathbb{V} \times \mathbb{V} \to \mathbb{R}$ is defined by
$$a(u, v;\boldsymbol{\mu})=\int_{\Omega} \nabla u\cdot \nabla v \ d\boldsymbol{x},$$
* the parametrized bilinear form $c(\cdot, \cdot; \boldsymbol{\mu}): \mathbb{V} \times \mathbb{V} \to \mathbb{R}$ is defined by
$$c(u, v;\boldsymbol{\mu})=\mu_0\int_{\Omega} \frac{1}{\mu_1}\big(\exp\{\mu_1u\} - 1\big)v \ d\boldsymbol{x},$$
* the parametrized linear form $f(\cdot; \boldsymbol{\mu}): \mathbb{V} \to \mathbb{R}$ is defined by
$$f(v; \boldsymbol{\mu})= \int_{\Omega}g(\boldsymbol{x}; \boldsymbol{\mu})v \ d\boldsymbol{x}.$$

The output of interest $s(\boldsymbol{\mu})$ is given by
$$s(\boldsymbol{\mu}) = \int_{\Omega} v \ d\boldsymbol{x}$$
is computed for each $\boldsymbol{\mu}$.

In [1]:
import os
import sys
sys.path.append('../../')

from mlnics import NN, Losses, Normalization, RONNData, IO, Training, ErrorAnalysis
from dolfin import *
from rbnics import *
import torch
import torch.nn as nn
import numpy as np
import time

torch.manual_seed(0)
np.random.seed(0)

### 3. Affine Decomposition 

For this problem the affine decomposition is straightforward:
$$a(u,v;\boldsymbol{\mu})=\underbrace{1}_{\Theta^{a}_0(\boldsymbol{\mu})}\underbrace{\int_{\Omega}\nabla u \cdot \nabla v \ d\boldsymbol{x}}_{a_0(u,v)},$$
$$c(u,v;\boldsymbol{\mu})=\underbrace{\mu_0}_{\Theta^{c}_0(\boldsymbol{\mu})}\underbrace{\int_{\Omega}\frac{1}{\mu_1}\big(\exp\{\mu_1u\} - 1\big)v \ d\boldsymbol{x}}_{c_0(u,v)},$$
$$f(v; \boldsymbol{\mu}) = \underbrace{100}_{\Theta^{f}_0(\boldsymbol{\mu})} \underbrace{\int_{\Omega}\sin(2\pi x_0)cos(2\pi x_1)v \ d\boldsymbol{x}}_{f_0(v)}.$$
We will implement the numerical discretization of the problem in the class
```
class NonlinearElliptic(NonlinearEllipticProblem):
```
by specifying the coefficients $\Theta^{a}_*(\boldsymbol{\mu})$, $\Theta^{c}_*(\boldsymbol{\mu})$ and $\Theta^{f}_*(\boldsymbol{\mu})$ in the method
```
    def compute_theta(self, term):
```
and the bilinear forms $a_*(u, v)$, $c_*(u, v)$ and linear forms $f_*(v)$ in
```
    def assemble_operator(self, term):
```

In [2]:
@ExactParametrizedFunctions()
class NonlinearElliptic(NonlinearEllipticProblem):

    # Default initialization of members
    def __init__(self, V, **kwargs):
        # Call the standard initialization
        NonlinearEllipticProblem.__init__(self, V, **kwargs)
        # ... and also store FEniCS data structures for assembly
        assert "subdomains" in kwargs
        assert "boundaries" in kwargs
        self.subdomains, self.boundaries = kwargs["subdomains"], kwargs["boundaries"]
        self.du = TrialFunction(V)
        self.u = self._solution
        self.v = TestFunction(V)
        self.dx = Measure("dx")(subdomain_data=self.subdomains)
        self.ds = Measure("ds")(subdomain_data=self.boundaries)
        # Store the forcing term expression
        self.f = Expression("sin(2*pi*x[0])*sin(2*pi*x[1])", element=self.V.ufl_element())
        # Customize nonlinear solver parameters
        self._nonlinear_solver_parameters.update({
            "linear_solver": "mumps",
            "maximum_iterations": 20,
            "report": True
        })

    # Return custom problem name
    def name(self):
        return "NonlinearEllipticExact"

    # Return theta multiplicative terms of the affine expansion of the problem.
    @compute_theta_for_derivatives
    def compute_theta(self, term):
        mu = self.mu
        if term == "a":
            theta_a0 = 1.
            return (theta_a0,)
        elif term == "c":
            theta_c0 = mu[0]
            return (theta_c0,)
        elif term == "f":
            theta_f0 = 100.
            return (theta_f0,)
        elif term == "s":
            theta_s0 = 1.0
            return (theta_s0,)
        else:
            raise ValueError("Invalid term for compute_theta().")

    # Return forms resulting from the discretization of the affine expansion of the problem operators.
    @assemble_operator_for_derivatives
    def assemble_operator(self, term):
        v = self.v
        dx = self.dx
        if term == "a":
            du = self.du
            a0 = inner(grad(du), grad(v)) * dx
            return (a0,)
        elif term == "c":
            u = self.u
            mu = self.mu
            c0 = (exp(mu[1] * u) - 1) / mu[1] * v * dx
            return (c0,)
        elif term == "f":
            f = self.f
            f0 = f * v * dx
            return (f0,)
        elif term == "s":
            s0 = v * dx
            return (s0,)
        elif term == "dirichlet_bc":
            bc0 = [DirichletBC(self.V, Constant(0.0), self.boundaries, 1)]
            return (bc0,)
        elif term == "inner_product":
            du = self.du
            x0 = inner(grad(du), grad(v)) * dx
            return (x0,)
        else:
            raise ValueError("Invalid term for assemble_operator().")


# Customize the resulting reduced problem
@CustomizeReducedProblemFor(NonlinearEllipticProblem)
def CustomizeReducedNonlinearElliptic(ReducedNonlinearElliptic_Base):
    class ReducedNonlinearElliptic(ReducedNonlinearElliptic_Base):
        def __init__(self, truth_problem, **kwargs):
            ReducedNonlinearElliptic_Base.__init__(self, truth_problem, **kwargs)
            self._nonlinear_solver_parameters.update({
                "report": True,
                "line_search": "wolfe"
            })

    return ReducedNonlinearElliptic

## 4. Main program

### 4.1. Read the mesh for this problem
The mesh was generated by the [data/generate_mesh.ipynb](data/generate_mesh.ipynb) notebook.

In [3]:
mesh = Mesh("data/square.xml")
subdomains = MeshFunction("size_t", mesh, "data/square_physical_region.xml")
boundaries = MeshFunction("size_t", mesh, "data/square_facet_region.xml")

### 4.2. Create Finite Element space (Lagrange P1)

In [4]:
V = FunctionSpace(mesh, "Lagrange", 1)

### 4.3. Allocate an object of the NonlinearElliptic class

In [5]:
problem = NonlinearElliptic(V, subdomains=subdomains, boundaries=boundaries)
mu_range = [(1., 10.0), (1., 10.0)]
problem.set_mu_range(mu_range)

### 4.4. Prepare reduction with a POD-Galerkin method

In [6]:
reduction_method = PODGalerkin(problem)
reduction_method.set_Nmax(20)
reduction_method.set_tolerance(1e-8)

### 4.5. Perform the offline phase

#### 4.5.1 Fit Reduction Method

In [7]:
reduction_method.initialize_training_set(10)
reduced_problem = reduction_method.offline()

=           NonlinearEllipticExact POD-Galerkin offline phase begins           =

###################################### 0 #######################################
truth solve for mu = (5.939321535345923, 7.436704297351775)
  0 SNES Function norm 1.510614e+00
  1 SNES Function norm 1.358684e+00
  2 SNES Function norm 1.220729e+00
  3 SNES Function norm 1.096053e+00
  4 SNES Function norm 8.987266e-01
  5 SNES Function norm 8.086250e-01
  6 SNES Function norm 1.974144e-01
  7 SNES Function norm 2.023093e-02
  8 SNES Function norm 2.560113e-04
  9 SNES Function norm 4.138558e-08
  10 SNES Function norm 4.116074e-15
PETSc SNES solver converged in 10 iterations with convergence reason 2.
update snapshots matrix

###################################### 1 #######################################
truth solve for mu = (6.4248703846447945, 5.903948646972072)
  0 SNES Function norm 1.510614e+00
  1 SNES Function norm 1.358956e+00
  2 SNES Function norm 1.132255e+00
  3 SNES Function norm 6.390213e-

In [8]:
class Net(nn.Module):
    def __init__(self, in_size, out_size):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(in_size, 80)
        self.fc2 = nn.Linear(80, 80)
        self.fc3 = nn.Linear(80, out_size)
    
    def forward(self, x):
        x = self.fc1(x)
        x = torch.tanh(x)
        x = self.fc2(x)
        x = torch.tanh(x)
        x = self.fc3(x)
        return x
    
class Net2(nn.Module):
    def __init__(self, in_size, out_size):
        super(Net2, self).__init__()
        self.fc1 = nn.Linear(in_size, 100)
        self.fc2 = nn.Linear(100, 100)
        self.fc3 = nn.Linear(100, out_size)
    
    def forward(self, x):
        x = self.fc1(x)
        x = torch.tanh(x)
        x = self.fc2(x)
        x = torch.tanh(x)
        x = self.fc3(x)
        return x

In [9]:
# Step 1
# Get nonlinear_terms, parameters, N0
nonlinear_terms = np.zeros((len(reduction_method.training_set), len(reduction_method.training_set), reduced_problem.N))
params = np.array(reduction_method.training_set)
N0 = lambda sol, param_idx: 0 # N0 identically 0

solutions = []
Basis_Matrix = np.array([v.vector()[:] for v in reduced_problem.basis_functions])

for mu in reduction_method.training_set:
    problem.set_mu(mu)
    solution = problem.solve()
    solutions.append(np.array(problem._solution.vector()[:]))

for i, mu in enumerate(reduction_method.training_set):
    problem.set_mu(mu)
    operator_form = problem.assemble_operator('c')[0]
    theta = problem.compute_theta('c')
    
    for j, solution in enumerate(solutions):
        problem._solution.vector()[:] = solution
        nonlinear_terms[i, j] = (theta * Basis_Matrix @ np.array(assemble(operator_form)[:]).reshape(-1, 1)).reshape(-1)
        
solutions_ = []
for sol in solutions:
    F = Function(V)
    F.vector()[:] = sol
    solutions_.append(np.array(reduced_problem.project(F).vector()[:]))
solutions = solutions_

In [10]:
# For creating validation set
reduction_method.initialize_testing_set(10)
test_mu = torch.tensor(reduction_method.testing_set)

nonlinear_terms_test = np.zeros((len(reduction_method.training_set), len(test_mu), reduced_problem.N))
params_test = np.array(test_mu)

solutions_test = []

for mu in reduction_method.testing_set:
    mu = tuple(mu)
    problem.set_mu(mu)
    solution = problem.solve()
    solutions_test.append(np.array(problem._solution.vector()[:]))

for i, mu in enumerate(reduction_method.training_set):
    problem.set_mu(mu)
    operator_form = problem.assemble_operator('c')[0]
    theta = problem.compute_theta('c')
    
    for j, solution in enumerate(solutions_test):
        problem._solution.vector()[:] = solution
        nonlinear_terms_test[i, j] = (theta * Basis_Matrix @ np.array(assemble(operator_form)[:]).reshape(-1, 1)).reshape(-1)
        
solutions_ = []
for sol in solutions_test:
    F = Function(V)
    F.vector()[:] = sol
    solutions_.append(np.array(reduced_problem.project(F).vector()[:]))
solutions_test = solutions_

  0 SNES Function norm 1.510614e+00
  1 SNES Function norm 1.358038e+00
  2 SNES Function norm 1.219136e+00
  3 SNES Function norm 1.093816e+00
  4 SNES Function norm 8.836064e-01
  5 SNES Function norm 6.609141e-01
  6 SNES Function norm 1.422366e-01
  7 SNES Function norm 1.078055e-02
  8 SNES Function norm 7.111511e-05
  9 SNES Function norm 3.066130e-09
  10 SNES Function norm 4.348480e-15
PETSc SNES solver converged in 10 iterations with convergence reason 2.
  0 SNES Function norm 1.510614e+00
  1 SNES Function norm 1.358655e+00
  2 SNES Function norm 1.220527e+00
  3 SNES Function norm 1.095594e+00
  4 SNES Function norm 9.723193e-01
  5 SNES Function norm 6.514664e-01
  6 SNES Function norm 2.100091e-01
  7 SNES Function norm 2.366904e-02
  8 SNES Function norm 3.729250e-04
  9 SNES Function norm 9.256090e-08
  10 SNES Function norm 6.930638e-15
PETSc SNES solver converged in 10 iterations with convergence reason 2.
  0 SNES Function norm 1.510614e+00
  1 SNES Function norm 1.3

In [11]:
chosen_parameter_indices = []
fixed_mu_networks = []

# Step 2
# 2a. Set mu_1
errors = np.zeros(params.shape[0])
for i, mu_i in enumerate(params):
    # compute error
    s = 0
    for j, mu_j in enumerate(params):
        s += np.sum((nonlinear_terms[i, j] - N0(solutions[j], i))**2)
    errors[i] = s / params.shape[0]

mu_1_idx = np.argmax(errors)
mu_1 = params[mu_1_idx]
chosen_parameter_indices.append(mu_1_idx)
print("max error:", np.max(errors))
print("mu_1 index:", mu_1_idx)

# 2b. Train Network_{mu_1}(u) to approximate Nonlinearity(u; mu_1)
print("\nTraining network to approximate nonlinearity...")
Network_mu_1 = Net(reduced_problem.N, reduced_problem.N)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(Network_mu_1.parameters(), lr=0.001)

normalization = Normalization.IdentityNormalization()

x_data = torch.tensor(np.array(solutions)).float()
y_data = normalization(torch.tensor(nonlinear_terms[mu_1_idx]).float().T).T

x_test = torch.tensor(np.array(solutions_test)).float()
y_test = normalization(torch.tensor(nonlinear_terms_test[mu_1_idx]).float().T).T

for epoch in range(25000):
    optimizer.zero_grad()
    output = Network_mu_1(x_data)
    loss = criterion(output, y_data)
    if epoch % 100 == 0:
        with torch.no_grad():
            validation_loss = criterion(Network_mu_1(x_test), y_test)
            print(epoch, loss.item(), validation_loss.item())
    loss.backward()
    optimizer.step()

Network_mu_1.eval()
fixed_mu_networks.append(Network_mu_1)

# 2c. Find theta_1_1(mu)
print("\nFinding theta...")
thetas = np.zeros((params.shape[0], 1))
for i, mu_i in enumerate(params):
    numerator = 0
    denominator = 0
    for j, mu_j in enumerate(params):
        net_u_mu = normalization(Network_mu_1(x_data[j].view(1, -1)).detach().T, normalize=False).T.numpy().reshape(-1)
        numerator += np.dot(nonlinear_terms[i, j], net_u_mu)
        denominator += np.dot(net_u_mu, net_u_mu)
        
    theta_1_1_i = numerator / denominator
    thetas[i] = theta_1_1_i
    print(thetas[i])

max error: 16095.443597988517
mu_1 index: 6

Training network to approximate nonlinearity...
0 1.0834641456604004 1.0743765830993652
100 0.054260533303022385 0.09078515321016312
200 0.02173236757516861 0.04830966144800186
300 0.006565386429429054 0.018969856202602386
400 0.0022162606474012136 0.0077117010951042175
500 0.0008678266312927008 0.0036088230554014444
600 0.0003536787407938391 0.0018797015072777867
700 0.00014087153249420226 0.0010939083294942975
800 5.523439176613465e-05 0.0007323495810851455
900 2.3502805561292917e-05 0.0005650792736560106
1000 1.2509191947174259e-05 0.00048223763587884605
1100 8.655510100652464e-06 0.0004342384054325521
1200 7.0808919190312736e-06 0.00040070718387141824
1300 6.2213871387939434e-06 0.00037385133327916265
1400 5.604948910331586e-06 0.0003507682413328439
1500 5.093724666949129e-06 0.0003303075791336596
1600 4.646192337531829e-06 0.0003119340108241886
1700 4.247407105140155e-06 0.00029531895415857434
1800 3.889918389177183e-06 0.00028022512560

16200 1.2189630069769919e-06 0.00019565221737138927
16300 3.212713181710569e-06 0.00022035407891962677
16400 7.73780448071193e-06 0.00022880050528328866
16500 1.6016958852560492e-06 0.00021149913663975894
16600 2.09604445444711e-06 0.000221257985685952
16700 1.0287746476933535e-07 0.00021812475461047143
16800 2.490840813607065e-07 0.00021620788902509958
16900 2.2142507987155113e-06 0.00020769172988366336
17000 2.267189756821608e-06 0.00022248877212405205
17100 1.3462421520671342e-05 0.0002679606550373137
17200 3.826355623459676e-06 0.00023525161668658257
17300 2.445314203214366e-05 0.00027542011230252683
17400 1.425345635652775e-05 0.00027233577566221356
17500 4.2247185660926334e-07 0.00024316586495842785
17600 1.3310745998751372e-06 0.00024348335864488035
17700 2.248537384730298e-05 0.0003341763513162732
17800 3.732002141987323e-06 0.0002925479784607887
17900 7.195211537691648e-07 0.00027015438536182046
18000 4.917939691040374e-07 0.0002794493339024484
18100 1.067939319909783e-05 0.00

In [12]:
def numpy_normalize(x):
    return normalization(torch.tensor(x.reshape(1, -1)).float().T).T.detach().numpy().reshape(-1)

In [13]:
matrices = []
maximum_error = np.max(errors)
#while maximum_error > 0.005:
for iteration in range(10):
    # Step 3
    # 3a. Set mu_2
    N1 = lambda sol, param_idx: sum([
        thetas[param_idx][i] * normalization(net(torch.tensor(sol).float().view(1, -1)).detach().T, normalize=False).T.numpy().reshape(-1)\
        for i, net in enumerate(fixed_mu_networks)
    ])
    
    

    errors = np.zeros(params.shape[0])
    for i, mu_i in enumerate(params):
        # compute error
        s = 0
        for j, mu_j in enumerate(params):
            s += np.sum((nonlinear_terms[i, j] - N1(solutions[j], i))**2)
        errors[i] = s / params.shape[0]
        
    mean_errors = np.zeros(params.shape[0])
    for i, mu_i in enumerate(params):
        mean_errors[i] = np.mean(np.abs((numpy_normalize(nonlinear_terms[i, i]) - numpy_normalize(N1(solutions[i], i))) / numpy_normalize(nonlinear_terms[i, i])))

    errors[np.array(chosen_parameter_indices)] = -1 # don't choose already chosen parameters again
    mu_2_idx = np.argmax(errors)
    mu_2 = params[mu_1_idx]
    chosen_parameter_indices.append(mu_2_idx)
    print("max error:", np.max(errors))
    maximum_error = np.max(errors)
    print("mean error:", np.mean(mean_errors))
    print("mu_2 index:", mu_2_idx)

    # 3b. Train Network_{mu_2}(u) to approximate Nonlinearity(u; mu_2)
    print("\nTraining network to approximate nonlinearity...")
    Network_mu_2 = Net(reduced_problem.N, reduced_problem.N)
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(Network_mu_2.parameters(), lr=0.001)

    y_data = normalization(torch.tensor(nonlinear_terms[mu_2_idx]).float().T).T
    y_test = normalization(torch.tensor(nonlinear_terms_test[mu_2_idx]).float().T).T

    for epoch in range(25000):
        optimizer.zero_grad()
        output = Network_mu_2(x_data)
        loss = criterion(output, y_data)
        if epoch % 100 == 0:
            with torch.no_grad():
                validation_loss = criterion(Network_mu_2(x_test), y_test)
                print(epoch, loss.item(), validation_loss.item())
        loss.backward()
        optimizer.step()

    Network_mu_2.eval()
    fixed_mu_networks.append(Network_mu_2)

    # 3c. Find theta_1_2(mu), theta_2_2(mu)
    print("\nFinding theta...")
    num_nets = len(fixed_mu_networks)
    thetas = np.zeros((params.shape[0], num_nets))
    for i, mu_i in enumerate(params):
        LHS_numerator = np.zeros((num_nets, num_nets))
        LHS_denominator = np.zeros((num_nets, num_nets))
        RHS_numerator = np.zeros((num_nets, 1))
        RHS_denominator = np.zeros((num_nets, 1))

        for j, mu_j in enumerate(params):
            nets_u_mu = [normalization(net(x_data[j].view(1, -1)).detach().T, normalize=False).T.numpy().reshape(-1) for net in fixed_mu_networks]

            for k1 in range(num_nets):
                RHS_numerator[k1] += np.dot(nonlinear_terms[i, j], nets_u_mu[k1])
                RHS_denominator[k1] += np.dot(nets_u_mu[k1], nets_u_mu[k1])
                for k2 in range(num_nets):
                    
                    LHS_numerator[k1, k2] += np.dot(nets_u_mu[k1], nets_u_mu[k2])
                    LHS_denominator[k1, k2] += np.dot(nets_u_mu[k1], nets_u_mu[k1])



        LHS = LHS_numerator / LHS_denominator
        RHS = RHS_numerator / RHS_denominator
        print(np.linalg.cond(LHS))
        matrices.append(LHS)
        thetas[i] = np.linalg.solve(LHS, RHS).reshape(-1)
        #print(thetas[i])

max error: 2.8587630347118154
mean error: 0.0019188886479241773
mu_2 index: 9

Training network to approximate nonlinearity...
0 1.2905722856521606 1.3876421451568604
100 0.03634921833872795 0.04115603491663933
200 0.01303731370717287 0.02760365419089794
300 0.005388358607888222 0.013707796111702919
400 0.0023958897218108177 0.00688276207074523
500 0.0011363144731149077 0.003642304567620158
600 0.0005496778176166117 0.0020097747910767794
700 0.00026013090973719954 0.0011486579896882176
800 0.00011679078306769952 0.0006878261920064688
900 4.954624091624282e-05 0.0004463270306587219
1000 2.068575849989429e-05 0.00032418378395959735
1100 9.392928404849954e-06 0.00026335223810747266
1200 5.2582081480068155e-06 0.0002319867053302005
1300 3.7520198929996695e-06 0.00021418002143036574
1400 3.1373770070786122e-06 0.0002025449794018641
1500 2.8119279704696964e-06 0.0001937917695613578
1600 2.5830008780758362e-06 0.00018647717661224306
1700 2.392292572039878e-06 0.00017997532268054783
1800 2.222

16200 1.1203279655092047e-06 0.00015716717462055385
16300 1.470933312930356e-07 0.00015374153736047447
16400 1.3073959962639492e-05 0.00015142638585530221
16500 7.176365102168347e-07 0.00016578448412474245
16600 3.669356374302879e-07 0.0001663212024141103
16700 1.0308664855074312e-07 0.00017040689999703318
16800 1.1968321587119135e-06 0.00017917371587827802
16900 3.1276440495275892e-06 0.00018373307830188423
17000 3.0270550723798806e-06 0.00019532488659024239
17100 5.842202881467529e-06 0.00017027001013047993
17200 6.702268819935853e-06 0.00018821479170583189
17300 2.4275202576973243e-06 0.00019823343609459698
17400 3.5535056213120697e-06 0.00019591751333791763
17500 9.143466741079465e-06 0.00021602795459330082
17600 5.759532541560475e-06 0.00017932752962224185
17700 2.4482478693244047e-06 0.0001936939952429384
17800 4.4359387629810954e-07 0.00021165226644370705
17900 7.274819040503644e-07 0.00021664751693606377
18000 5.143437533661199e-07 0.00021664831729140133
18100 4.219170477881562

6900 1.1885949788847938e-05 7.143421680666506e-05
7000 1.5728910511825234e-06 6.252441380638629e-05
7100 1.173345935967518e-06 6.070595554774627e-05
7200 1.2958486877323594e-05 7.466428360203281e-05
7300 2.4387793473579222e-06 5.543716542888433e-05
7400 9.41850885283202e-06 5.891578257433139e-05
7500 4.326064754422987e-06 5.932018757448532e-05
7600 1.8112898032995872e-05 7.109803118510172e-05
7700 1.157748670266301e-06 5.009618689655326e-05
7800 4.88817022414878e-05 0.00010500643838895485
7900 4.77516878163442e-05 9.044136822922155e-05
8000 4.39533550888882e-06 4.828819146496244e-05
8100 8.341949069290422e-06 4.523642564890906e-05
8200 6.4175314946623985e-06 4.515306500252336e-05
8300 1.6316273558913963e-06 3.8218455301830545e-05
8400 5.194822847442992e-07 3.617807669797912e-05
8500 4.7429639948859403e-07 3.5288558137835935e-05
8600 5.3573194236378185e-06 3.605800156947225e-05
8700 5.467457526719954e-07 3.204926906619221e-05
8800 7.470022865163628e-06 3.9314396417466924e-05
8900 2.8669

23200 7.253982658994573e-08 5.427237283583963e-06
23300 1.4009669939696323e-05 1.8646054741111584e-05
23400 8.982202359675284e-08 6.14283135291771e-06
23500 3.299035933324035e-09 5.964521278656321e-06
23600 2.034689572383286e-08 5.908568255108548e-06
23700 3.6618018839362776e-06 9.722734830575064e-06
23800 3.404237247650599e-07 5.972403869236587e-06
23900 6.52215157970204e-06 1.007141236186726e-05
24000 2.6848953282865295e-08 6.133206170488847e-06
24100 1.0784020560095087e-05 2.2753863959223963e-05
24200 4.5589473529616953e-07 6.103377472754801e-06
24300 7.024637511676701e-07 8.29086457088124e-06
24400 5.186439011595212e-07 7.162854672060348e-06
24500 4.904386514681391e-07 6.727676463924581e-06
24600 9.308861990575679e-06 1.2553644410218112e-05
24700 1.2501483979576733e-05 1.716778388072271e-05
24800 1.100850113289198e-06 8.771492503001355e-06
24900 1.0634266089937228e-07 7.634070243511815e-06

Finding theta...
326561.6985037932
326561.6985037932
326561.6985037932
326561.6985037932
326

14000 1.290111981688824e-07 6.486080224021862e-07
14100 3.622394117996919e-08 5.010363679502916e-07
14200 3.5429998490599246e-08 5.050745244261634e-07
14300 1.619840617195223e-07 6.284504365794419e-07
14400 3.1369431781058665e-06 3.7970542052789824e-06
14500 1.5635981981176883e-05 1.5355833966168575e-05
14600 7.502298444705957e-07 1.3213040119808284e-06
14700 2.1150931672764273e-07 5.831603857586742e-07
14800 3.7191949786574696e-07 9.33349156184704e-07
14900 9.060612683242653e-06 1.027917824103497e-05
15000 1.0241348718409427e-05 1.0538487003941555e-05
15100 5.320817422216351e-07 1.1387959375497303e-06
15200 1.8749171886156546e-06 2.361413635298959e-06
15300 1.711622076072672e-06 2.4738938009249978e-06
15400 4.5601572651321476e-07 7.283875333996548e-07
15500 3.760825620702235e-06 3.945719527109759e-06
15600 3.815906438831007e-06 3.645535116447718e-06
15700 9.809685934669687e-07 1.369355913993786e-06
15800 3.8374614632630255e-06 4.6708678382856306e-06
15900 5.162185061635682e-07 9.84895

...

200 5.8969762903871015e-06 1.0415564247523434e-05
300 3.318230028526159e-06 5.515802513400558e-06
400 2.0497966488619568e-06 3.1963145374902524e-06
500 1.2571088063850766e-06 1.855496066127671e-06
600 7.609937142660783e-07 1.059146597981453e-06
700 4.55496973472691e-07 5.964174079053919e-07
800 2.7150321102453745e-07 3.3642078278717236e-07
900 1.631293997661487e-07 1.9593984745824855e-07
1000 1.0058619892561182e-07 1.23621035186261e-07
1100 6.505310778948115e-08 8.882445001745509e-08
1200 4.507962358957229e-08 7.378319821782497e-08
1300 3.385618541074109e-08 6.867968949109127e-08
1400 2.7413387115871046e-08 6.810675046153847e-08
1500 2.3578563101978034e-08 6.93219348590901e-08
1600 2.1137854133712608e-08 7.092789644502773e-08
1700 1.9444966525838936e-08 7.231219001369027e-08
1800 1.8179457939027088e-08 7.327833628778535e-08
1900 1.7163166887712578e-08 7.3723768423406e-08
2000 1.630153789733413e-08 7.374309518581867e-08
2100 1.5553510479549004e-08 7.337304452903481e-08
2200 1.4884812493

16600 8.189697950911068e-07 8.193848657356284e-07
16700 1.1135280431062711e-07 1.1514428166492507e-07
16800 9.328849955636542e-06 9.357059752801433e-06
16900 5.360272439247638e-07 5.380741185945226e-07
17000 7.418009886350774e-07 7.667406407563249e-07
17100 5.076253728475422e-07 5.172020678401168e-07
17200 1.5690462532802485e-05 1.5669311324018054e-05
17300 1.260590465790301e-06 1.2932002846355317e-06
17400 2.406409976174473e-06 2.348894213355379e-06
17500 9.592530858526516e-08 1.0491826429870343e-07
17600 3.98137308366131e-06 3.904953700839542e-06
17700 2.9964206987642683e-05 2.9810991691192612e-05
17800 3.3000760595314205e-06 3.3558530958543997e-06
17900 1.0649332580214832e-05 1.068834444595268e-05
18000 3.5639598081615986e-06 3.544294031598838e-06
18100 4.07376774091972e-06 4.1438565858697984e-06
18200 2.4693679279153002e-06 2.4755618142080493e-06
18300 1.2377792700135615e-05 1.2504400729085319e-05
18400 5.8951418395736255e-06 5.821736976940883e-06
18500 4.4434779056246043e-07 4.429

7300 5.797412450192496e-05 8.220272866310552e-05
7400 5.104396336719219e-07 2.8304659281275235e-05
7500 5.845204213983379e-07 2.7740283258026466e-05
7600 9.331353680863685e-07 2.8608699722099118e-05
7700 5.358278372114e-07 2.77132658084156e-05
7800 8.987732144305483e-06 3.842468504444696e-05
7900 1.2529866353361285e-06 2.6426754629937932e-05
8000 5.992993465042673e-06 2.6165496819885448e-05
8100 8.660887942824047e-06 3.8208159821806476e-05
8200 8.655505894239468e-07 2.685794788703788e-05
8300 3.302727918708115e-06 3.021452903340105e-05
8400 1.3628074157168157e-05 3.658243440440856e-05
8500 1.5341145171987591e-06 2.6900526790996082e-05
8600 3.6137166716798674e-06 3.0363369660335593e-05
8700 4.087190177415323e-07 2.317731014045421e-05
8800 5.1884205731767e-07 2.3218362912302837e-05
8900 2.678637429198716e-06 2.223789124400355e-05
9000 4.8533803465034e-07 2.1909789211349562e-05
9100 1.311369032919174e-06 2.2235240976442583e-05
9200 3.976681000494864e-06 2.3015603801468387e-05
9300 1.29974

23600 4.5053849540011015e-09 8.094664849522815e-07
23700 4.191008073917146e-08 8.282851240437594e-07
23800 5.5582479063787105e-08 9.815262274059933e-07
23900 3.4035908811347326e-06 3.864166501443833e-06
24000 1.8563208641353413e-06 2.5324750367872184e-06
24100 1.0540820767346304e-06 2.1397913769760635e-06
24200 8.231663173319248e-07 1.3266852647575433e-06
24300 9.287017746828496e-06 9.406077879248187e-06
24400 1.7254749764106236e-05 1.8131198885384947e-05
24500 3.8369086041711853e-07 1.244939312528004e-06
24600 1.6519719565621926e-06 2.6343379886384355e-06
24700 2.9302789243956795e-06 2.7140551992488327e-06
24800 4.931616217618284e-07 1.4154504697216908e-06
24900 3.0419585073104827e-07 1.0128220537808375e-06

Finding theta...
22630919479.06558
22630919479.06558
22630919479.06558
22630919479.06558
22630919479.06558
22630919479.06558
22630919479.06558
22630919479.06558
22630919479.06558
22630919479.06558
max error: -1.0
mean error: 0.00045474278085748664
mu_2 index: 0

Training network t

14400 5.168032544133894e-07 5.660483111569192e-06
14500 4.7218412646543584e-07 5.483776931214379e-06
14600 1.1619569704635069e-05 2.18563036469277e-05
14700 7.585616458527511e-06 1.3641576515510678e-05
14800 8.419514415436424e-06 1.322197203990072e-05
14900 5.833640443597687e-06 1.3259154911793303e-05
15000 6.923200544406427e-06 1.330680515820859e-05
15100 9.781682820175774e-06 1.32257582663442e-05
15200 1.9150591015204554e-06 5.29170893059927e-06
15300 1.139311098086182e-05 1.4902670045557898e-05
15400 1.785729608627662e-07 4.922748303215485e-06
15500 7.594752560180495e-07 5.52885876459186e-06
15600 2.5268900571973063e-07 4.827041721000569e-06
15700 1.1753847502404824e-05 1.6436011719633825e-05
15800 1.1787775292759761e-05 1.3522212611860596e-05
15900 7.755080559945782e-07 4.638399786927039e-06
16000 7.671548701182473e-06 1.3560815204982646e-05
16100 4.7324411411864276e-07 4.600070496962871e-06
16200 1.253845653081953e-06 3.94888502341928e-06
16300 5.417513875727309e-06 9.189311640511

In [19]:
matrices[0], params[6], params[9]

(array([[1.        , 0.80902721],
        [1.23544325, 1.        ]]),
 array([6.11240105, 9.33036974]),
 array([8.00341076, 8.83010933]))

In [15]:
matrices2 = []

nonlinear_terms_test2 = np.zeros((len(params_test), len(params_test), reduced_problem.N))

solutions_test2 = []

for mu in reduction_method.testing_set:
    problem.set_mu(mu)
    solution = problem.solve()
    solutions_test2.append(np.array(problem._solution.vector()[:]))

for i, mu in enumerate(reduction_method.testing_set):
    problem.set_mu(mu)
    operator_form = problem.assemble_operator('c')[0]
    theta = problem.compute_theta('c')
    
    for j, solution in enumerate(solutions_test2):
        problem._solution.vector()[:] = solution
        nonlinear_terms_test2[i, j] = (theta * Basis_Matrix @ np.array(assemble(operator_form)[:]).reshape(-1, 1)).reshape(-1)

solutions_ = []
for sol in solutions_test2:
    F = Function(V)
    F.vector()[:] = sol
    solutions_.append(np.array(reduced_problem.project(F).vector()[:]))
solutions_test2 = solutions_

x_test = torch.tensor(np.array(solutions_test2)).float()

thetas_test = np.zeros((params_test.shape[0], num_nets))
for i, mu_i in enumerate(params_test):
    LHS_numerator = np.zeros((num_nets, num_nets))
    LHS_denominator = np.zeros((num_nets, num_nets))
    RHS_numerator = np.zeros((num_nets, 1))
    RHS_denominator = np.zeros((num_nets, 1))

    for j, mu_j in enumerate(params_test):
        nets_u_mu = [normalization(net(x_test[j].view(1, -1)).detach().T, normalize=False).T.numpy().reshape(-1) for net in fixed_mu_networks]

        for k1 in range(num_nets):
            RHS_numerator[k1] += np.dot(nonlinear_terms_test2[i, j], nets_u_mu[k1])
            RHS_denominator[k1] += np.dot(nets_u_mu[k1], nets_u_mu[k1])
            for k2 in range(num_nets):
                LHS_numerator[k1, k2] += np.dot(nets_u_mu[k1], nets_u_mu[k2])
                LHS_denominator[k1, k2] += np.dot(nets_u_mu[k1], nets_u_mu[k1])



    LHS = LHS_numerator / LHS_denominator
    RHS = RHS_numerator / RHS_denominator
    print(np.linalg.cond(LHS))
    matrices2.append(LHS)
    thetas_test[i] = np.linalg.solve(LHS, RHS).reshape(-1)

8275375.706417732
8275375.706417732
8275375.706417732
8275375.706417732
8275375.706417732
8275375.706417732
8275375.706417732
8275375.706417732
8275375.706417732
8275375.706417732


In [ ]:
# Final step: train interpolation for theta
theta_net = Net2(params.shape[1], thetas.shape[1])
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(theta_net.parameters(), lr=0.001)

x_data = torch.tensor(params).float()
y_data = torch.tensor(thetas).float()
x_test = torch.tensor(params_test).float()
y_test = torch.tensor(thetas_test).float()

x_normalization = Normalization.MinMaxNormalization(input_normalization=True)
y_normalization = Normalization.MinMaxNormalization()

x_normalization2 = Normalization.MinMaxNormalization(input_normalization=True)
y_normalization2 = Normalization.MinMaxNormalization()

x_data = x_normalization(x_data)
x_test = x_normalization2(x_test)
y_data = y_normalization(y_data.T).T
y_test = y_normalization2(y_test.T).T

theta_net.train()

for epoch in range(25000):
    optimizer.zero_grad()
    output = theta_net(x_data)
    loss = criterion(output, y_data)
    if epoch % 1 == 0:
        with torch.no_grad():
            output_test = theta_net(x_test)
            loss_test = criterion(output_test, y_test)
            print(epoch, loss.item(), loss_test.item())
    loss.backward()
    optimizer.step()

theta_net.eval()

In [ ]:
LHS

In [ ]:
npsol = np.array(solutions)
npsol_test = np.array(solutions_test2)
for i in range(npsol.shape[1]):
    plt.clf()
    plt.hist(npsol[:, i])
    plt.show()
    plt.clf()
    plt.hist(npsol_test[:, i])
    plt.show()

In [ ]:
for i in range(2):
    plt.clf()
    plt.hist(x_data.detach().numpy()[:, i])
    plt.show()
    plt.clf()
    plt.hist(x_test.detach().numpy()[:, i])
    plt.show()

In [ ]:
for i in range(num_nets):
    plt.clf()
    plt.hist(y_data.detach().numpy()[:, i])
    plt.show()
    plt.clf()
    plt.hist(y_test.detach().numpy()[:, i])
    plt.show()

In [ ]:
aa = x_data.detach().numpy()
bb = y_data.detach().numpy()
for i in range(bb.shape[1]):
    plt.clf()
    plt.scatter(aa[:, 0], aa[:, 1], c=bb[:, i])
    plt.colorbar()
    plt.show()

In [ ]:
aa = x_test.detach().numpy()
bb = y_test.detach().numpy()
for i in range(bb.shape[1]):
    plt.clf()
    plt.scatter(aa[:, 0], aa[:, 1], c=bb[:, i])
    plt.colorbar()
    plt.show()

In [ ]:
for i in range(bb.shape[1]):
    print(np.max(bb[:, i]))

# Perhaps a problem with sum approximation of integral with too few samples???

In [ ]:
plt.scatter(params_test[:, 0], params_test[:, 1])

In [ ]:
plt.scatter(params[:, 0], params[:, 1])

In [ ]:
import matplotlib.pyplot as plt
# look at best thetas in parameter space
for i in range(thetas.shape[1]):
    plt.clf()
    plt.scatter(params_test[:, 0], params_test[:, 1], c=thetas_test[:, i])
    plt.colorbar()
    plt.show()

In [ ]:
for i in range(thetas.shape[1]):
    plt.clf()
    plt.scatter(params[:, 0], params[:, 1], c=thetas[:, i])
    plt.colorbar()
    plt.show()

In [ ]:
# plot together!
plt.clf()
for i in range(thetas.shape[1]):
    p0 = np.concatenate([params[:, 0], params_test[:, 0]])
    p1 = np.concatenate([params[:, 1], params_test[:, 1]])
    tc = np.concatenate([thetas[:, i], thetas_test[:, i]])
    plt.scatter(p0, p1, c=tc)
    plt.colorbar()
    plt.show()

In [ ]:
for i in range(thetas.shape[1]):
    plt.scatter(params[:, 0], params[:, 1], c=y_normalization(theta_net(x_data).T, normalize=False).T.detach().numpy()[:, i])
    plt.colorbar()
    plt.show()

In [ ]:
def NonlinearityApprox(u, mu):
    thetas_ = theta_net(torch.tensor(mu).float().view(1, -1)).detach().numpy().reshape(-1, 1)
    net_evals = np.array([normalization(net(torch.tensor(u).float().view(1, -1)).T, normalize=False).T.detach().numpy().reshape(-1) for net in fixed_mu_networks])
    return np.sum(thetas_ * net_evals, axis=0)

def NonlinearityApprox2(u, mu):
    thetas_ = theta_net(torch.tensor(mu).float().view(1, -1)).view(-1).detach()
    s = 0
    for i, net in enumerate(fixed_mu_networks):
        s += thetas_[i] * normalization(net(u.view(1, -1)).T, normalize=False).T.view(-1)
    return s

In [ ]:
import matplotlib.pyplot as plt

approximations = []
for p in params_test:
    reduced_problem.set_mu(tuple(p))
    solution = np.array(reduced_problem.solve().vector()[:]).reshape(-1)
    approx = NonlinearityApprox(solution, p)
    approximations.append(approx)

for component in range(approximations[0].shape[0]):
    idx = np.arange(params_test.shape[0])[(params_test[:, 0] > 2) & (params_test[:, 1] > 2)]
    plt.scatter(params_test[idx, 0], params_test[idx, 1], c=np.array(approximations)[idx, component])
    plt.colorbar()
    plt.show()

In [ ]:
nonlinear_terms_matching = []
for i in range(params.shape[0]):
    nonlinear_terms_matching.append(nonlinear_terms[i, i])
nonlinear_terms_matching = np.array(nonlinear_terms_matching)

for component in range(approximations[0].shape[0]):
    plt.scatter(params[:, 0], params[:, 1], c=np.array(nonlinear_terms_matching)[:, component])
    plt.colorbar()
    plt.show()

In [ ]:
for component in range(approximations[0].shape[0]):
    plt.scatter(params[:, 0], params[:, 1], c=np.abs((np.array(nonlinear_terms_matching)[:, component] - np.array(approximations)[:, component])/np.array(nonlinear_terms_matching)[:, component]))
    plt.colorbar()
    plt.show()
    errors = np.abs((np.array(nonlinear_terms_matching)[:, component] - np.array(approximations)[:, component])/np.array(nonlinear_terms_matching)[:, component])
    print(np.mean(errors), np.min(errors), np.median(errors), np.max(errors))

In [ ]:
import matplotlib.pyplot as plt

plt.scatter(params[:, 0], params[:, 1])
plt.show()

In [ ]:
for i in range(nonlinear_terms_test.shape[0]):
    for j in range(nonlinear_terms_test.shape[1]):
        #print(nonlinear_terms_test[i, j])
        #print(NonlinearityApprox(solutions_test[j], params[i]))
        print(params[i], params_test[j],
              np.linalg.norm(nonlinear_terms_test[i, j] - NonlinearityApprox(solutions_test[j], params[i])),
              np.linalg.norm(nonlinear_terms_test[i, j] - NonlinearityApprox(solutions_test[j], params[i])) / np.linalg.norm(nonlinear_terms_test[i, j])
             )
    print("")

In [ ]:
ss = 0
for i in range(nonlinear_terms_test.shape[0]):
    s = 0
    for j in range(nonlinear_terms_test.shape[1]):
        s += np.linalg.norm(nonlinear_terms_test[i, j] - NonlinearityApprox(solutions_test[j], params[i]))
    print(s / nonlinear_terms_test.shape[1])
    ss += s
print("\n")
print(ss / nonlinear_terms_test.shape[0] / nonlinear_terms_test.shape[1])

In [ ]:
ss = 0
for i in range(nonlinear_terms_test.shape[0]):
    s = 0
    for j in range(nonlinear_terms_test.shape[1]):
        s += np.linalg.norm(nonlinear_terms_test[i, j] - NonlinearityApprox(solutions_test[j], params[i])) / np.linalg.norm(nonlinear_terms_test[i, j])
    print(s / nonlinear_terms_test.shape[1])
    ss += s
print("\n")
print(ss / nonlinear_terms_test.shape[0] / nonlinear_terms_test.shape[1])

In [ ]:
for i, mu in enumerate(reduction_method.testing_set):
    problem.set_mu(mu)
    solution = problem.solve()
    solution = np.array(problem._solution.vector()[:])
    
    operator_form = problem.assemble_operator('c')[0]
    theta = problem.compute_theta('c')
    problem._solution.vector()[:] = solution
    
    nonlinear_term_i = (theta * Basis_Matrix @ np.array(assemble(operator_form)[:]).reshape(-1, 1)).reshape(-1)
    nonlinear_term_approx_i = NonlinearityApprox(solutions_test[i], mu)
    
    print(np.linalg.norm(nonlinear_term_i - nonlinear_term_approx_i), np.linalg.norm(nonlinear_term_i - nonlinear_term_approx_i)/np.linalg.norm(nonlinear_term_i))

In [ ]:
class Approx_PINN_Loss(Losses.RONN_Loss_Base):
    """
    PINN_Loss

    ronn: object of type RONN

    RETURNS: loss function loss_fn(parameters, reduced order coefficients)
    """
    def __init__(self, ronn, normalization=None, beta=1., mu=None):
        super(Approx_PINN_Loss, self).__init__(ronn, mu)
        self.operators = None
        self.proj_snapshots = None
        self.T0_idx = None
        self.normalization = normalization
        if self.normalization is None:
            self.normalization = IdentityNormalization()

        self.beta = beta

        # if time dependent, we need the neural net to compute time derivative
        self.time_dependent = False

    def name(self):
        return "Approx_PINN"

    def _compute_operators(self):
        self.operators_initialized = True

        #self.operators = self.ronn.get_operator_matrices(self.mu)
        self.operators = self.ronn.get_reduced_operator_matrices(self.mu)

        if not self.normalization.initialized:
            self.normalization(self.ronn.get_projected_snapshots())

    def set_mu(self, mu):
        self.mu = mu
        self.operators_initialized = False

    def __call__(self, **kwargs):
        pred = kwargs["prediction_no_snap"]
        if not self.operators_initialized:
            self._compute_operators()

        pred = self.normalization(pred.T, normalize=False).T

        ##### 1st equation in system #####
        res1 = 0.0

        # these two could be combined when both not None
        if 'f' in self.operators:
            res1 -= self.operators['f']
        if 'c' in self.operators:
            for i, mu in enumerate(kwargs["input_normalization"](kwargs["normalized_mu"], normalize=False)):                
                mu = np.array(mu)
                #sol = pred[i].detach().numpy().reshape(-1)
                #C = NonlinearityApprox(sol, mu).reshape(-1, 1)
                sol = pred[i].float()
                C = NonlinearityApprox2(sol, mu).view(-1, 1).double().detach()

                # set element of self.operators['c']
                #self.operators['c'][i] = torch.tensor(C).double()[None, :, :]
                self.operators['c'][i] = C[None, :, :]

            res1 += self.operators['c']
        
        if 'a' in self.operators:
            res1 += torch.matmul(self.operators['a'], pred[:, :, None].double())

        loss1 = torch.mean(torch.sum(res1**2, dim=1)) if type(res1) is not float else res1
        if self.ronn.problem.dirichlet_bc_are_homogeneous:
            boundary_condition_loss = 0
        else:
            boundary_condition_loss = torch.mean((pred[:, 0] - 1.)**2)

        self.value = loss1 + self.beta*boundary_condition_loss

        return self.value

    def reinitialize(self, mu):
        normalization = self.normalization
        beta = self.beta
        return Approx_PINN_Loss(self.ronn, normalization, beta, mu)

In [ ]:
torch.manual_seed(42)

input_normalization_pinn = Normalization.MinMaxNormalization(input_normalization=True)
output_normalization_pinn = Normalization.MinMaxNormalization()

pinn_net  = NN.RONN("Approx_PINN", problem, reduction_method, n_hidden=2, n_neurons=40)
pinn_loss = Approx_PINN_Loss(pinn_net, output_normalization_pinn)
data      = RONNData.RONNDataLoader(pinn_net, validation_proportion=0.2, 
                                    num_without_snapshots=10000)
optimizer = torch.optim.Adam(pinn_net.parameters(), lr=0.001)
scheduler = None

pinn_trainer = Training.PINNTrainer(
    pinn_net, data, pinn_loss, optimizer, scheduler,
    input_normalization_pinn, num_epochs=10000, print_every=1
)

loaded, starting_epoch = IO.initialize_parameters(
    pinn_net, data, pinn_trainer, optimizer
)

In [ ]:
pinn_trainer.train()

In [ ]:
fig, ax = Training.plot_loss(pinn_trainer, pinn_net)

#### 4.5.2 Train PINN

Given a training set $X_{PINN} = (\boldsymbol{\mu}^{(1)}, \dots, \boldsymbol{\mu}^{(n)})$ of parameters for the PDE, we train a Physics-Informed Neural Network (PINN) $\operatorname{N}_W(\boldsymbol{\mu})$ dependent on the weights and biases $W$ of the network to minimize the loss function

$$L_{PINN}(X_{PINN}; W) = \frac1n \sum_{i=1}^n \left\|A(\boldsymbol{\mu^{(i)}}) \operatorname{N}_W(\boldsymbol{\mu}^{(i)}) - \boldsymbol{f}(\boldsymbol{\mu}^{(i)}) + \boldsymbol{c}(\boldsymbol{\mu}^{(i)})\right\|_2^2$$

over $W$, where for a given $\boldsymbol{\mu}$, $A(\boldsymbol{\mu})$ is the assembled matrix corresponding to the bilinear form $a$, $\boldsymbol{f}(\boldsymbol{\mu})$ is the assembled vector corresponding to the linear form $f$, and $\boldsymbol{c}(\boldsymbol{\mu})$ is a vector corresponding to the nonlinear form $c$.

In [ ]:
input_normalization_pinn = Normalization.MinMaxNormalization(input_normalization=True)
output_normalization_pinn = Normalization.MinMaxNormalization(input_normalization=False)

pinn_net  = NN.RONN("PINN", problem, reduction_method, n_hidden=2, n_neurons=40)
pinn_loss = Losses.PINN_Loss(pinn_net, output_normalization_pinn)
data      = RONNData.RONNDataLoader(pinn_net, validation_proportion=0.2, 
                                    num_without_snapshots=100)
optimizer = torch.optim.Adam(pinn_net.parameters(), lr=0.001)
scheduler = None

pinn_trainer = Training.PINNTrainer(
    pinn_net, data, pinn_loss, optimizer, scheduler,
    input_normalization_pinn, num_epochs=10000, print_every=1
)

loaded, starting_epoch = IO.initialize_parameters(
    pinn_net, data, pinn_trainer, optimizer
)

In [ ]:
start = time.time()
pinn_trainer.train()
end = time.time()
print(end - start)

In [ ]:
fig, ax = Training.plot_loss(pinn_trainer, pinn_net)

#### 4.5.3 Train PDNN

Given a training set $X_{PDNN} = ((\boldsymbol{\mu}^{(1)}, \operatorname{HF}(\boldsymbol{\mu}^{(1)})), \dots, (\boldsymbol{\mu}^{(n)}, \operatorname{HF}(\boldsymbol{\mu}^{(n)})))$ of parameter and high fidelity solution pairs for the PDE, we train a Projection-Driven Neural Network (PDNN) $\operatorname{N}_W(\boldsymbol{\mu})$ dependent on the weights and biases $W$ of the network to minimize the loss function
$$L_{PDNN}(X_{PDNN}; W) = \frac1n \sum_{i=1}^n \|\operatorname{N}_W(\boldsymbol{\mu}^{(i)}) - \tilde{\operatorname{HF}}(\boldsymbol{\mu}^{(i)})\|_2^2,$$
where for a given $\boldsymbol{\mu}$, $\tilde{\operatorname{HF}}(\boldsymbol{\mu})$ is the projection of $\operatorname{HF}(\boldsymbol{\mu})$ onto the reduced order solution space.

In [ ]:
input_normalization_pdnn = Normalization.StandardNormalization(input_normalization=True)
output_normalization_pdnn = Normalization.StandardNormalization()

pdnn_net  = NN.RONN("PDNN", problem, reduction_method, n_hidden=2, n_neurons=40)
pdnn_loss = Losses.PDNN_Loss(pdnn_net, output_normalization_pdnn)
data      = RONNData.RONNDataLoader(pdnn_net, validation_proportion=0.2)
optimizer = torch.optim.Adam(pdnn_net.parameters(), lr=0.001)

pdnn_trainer = Training.PDNNTrainer(
    pdnn_net, data, pdnn_loss, optimizer,
    input_normalization_pdnn, num_epochs=10000
)

loaded, starting_epoch = IO.initialize_parameters(
    pdnn_net, data, pdnn_trainer, optimizer
)

In [ ]:
pdnn_trainer.train()

In [ ]:
fig, ax = Training.plot_loss(pdnn_trainer, pdnn_net)

#### 4.5.4 Train PRNN

We train a Physics-Reinforced Neural Network (PRNN) $N_W(\boldsymbol{\mu})$ dependnent on the weights and biases $W$ of the network to minimize the loss function

$$L_{PRNN}(X_{PINN}, X_{PDNN}; W) = L_{PINN}(X_{PINN}; W) + \omega L_{PDNN}(X_{PDNN}; W),$$

where $\omega$ is a scaling parameter which can be chosen freely.

In [ ]:
input_normalization_prnn = Normalization.StandardNormalization(input_normalization=True)
output_normalization_prnn = Normalization.StandardNormalization()

omega = 1.
prnn_net  = NN.RONN(f"PRNN_{omega}", problem, reduction_method, n_hidden=2, n_neurons=40)
prnn_loss = Losses.PRNN_Loss(prnn_net, output_normalization_prnn, omega=omega)
data      = RONNData.RONNDataLoader(prnn_net, validation_proportion=0.2,
                                    num_without_snapshots=100)
optimizer = torch.optim.Adam(prnn_net.parameters(), lr=0.001)

prnn_trainer = Training.PRNNTrainer(
    prnn_net, data, prnn_loss, optimizer,
    input_normalization_prnn, num_epochs=1000
)

loaded, starting_epoch = IO.initialize_parameters(
    prnn_net, data, prnn_trainer, optimizer
)

In [ ]:
prnn_trainer.train()

In [ ]:
fig, ax = Training.plot_loss(prnn_trainer, prnn_net, separate=True)

### 4.6. Perform an error analysis

#### 4.6.1 Reduction Method Error Analysis

In [ ]:
reduction_method.initialize_testing_set(50)
#reduction_method.error_analysis()

#### 4.6.2 PINN Error Analysis

In [ ]:
test_mu = torch.tensor(reduction_method.testing_set)

In [ ]:
_ = ErrorAnalysis.error_analysis_fixed_net(
    pinn_net, test_mu, input_normalization_pinn, output_normalization_pinn, relative=True
)

In [ ]:
ErrorAnalysis.plot_solution_difference(
    pinn_net, (5., 5.), input_normalization_pinn, output_normalization_pinn, colorbar=True
)

#### 4.6.3 PDNN Error Analysis

In [ ]:
_ = ErrorAnalysis.error_analysis_fixed_net(
    pdnn_net, test_mu, input_normalization_pdnn, output_normalization_pdnn, relative=False
)

In [ ]:
ErrorAnalysis.plot_solution_difference(
    pdnn_net, (5., 5.), input_normalization_pdnn, output_normalization_pdnn
)

#### 4.6.4 PRNN Error Analysis

In [ ]:
_ = ErrorAnalysis.error_analysis_fixed_net(
    prnn_net, test_mu, input_normalization_prnn, output_normalization_prnn, relative=False
)

In [ ]:
ErrorAnalysis.plot_solution_difference(
    prnn_net, (5., 5.), input_normalization_prnn, output_normalization_prnn
)

#### 4.6.5 Neural Network Error Comparison

In [ ]:
nets = dict()
nets["pinn_net"] = pinn_net
nets["pdnn_net"] = pdnn_net
nets["prnn_net"] = prnn_net

input_normalizations = dict()
input_normalizations["pinn_net"] = input_normalization_pinn
input_normalizations["pdnn_net"] = input_normalization_pdnn
input_normalizations["prnn_net"] = input_normalization_prnn

output_normalizations = dict()
output_normalizations["pinn_net"] = output_normalization_pinn
output_normalizations["pdnn_net"] = output_normalization_pdnn
output_normalizations["prnn_net"] = output_normalization_prnn

_ = ErrorAnalysis.error_analysis_by_network(
    nets, test_mu, input_normalizations, output_normalizations, relative=False
)

### 4.7. Perform a speedup analysis

In [ ]:
reduction_method.speedup_analysis()